<a href="https://colab.research.google.com/github/mysticaven/EDGE_CNN_ESP32/blob/main/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # <<< ADD THIS
    transforms.Resize((32, 32)),
    transforms.ToTensor(),                         # [0,1]
    transforms.Lambda(lambda x: torch.round(x * 128) / 128)
])



In [4]:
train_ds = datasets.ImageFolder("/content/drive/MyDrive/dataset (1)/train", transform=transform)
val_ds   = datasets.ImageFolder("/content/drive/MyDrive/dataset (1)/val", transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32)

print("Classes:", train_ds.classes)
# should be ['not_tiger', 'tiger']


Classes: ['non_tiger', 'tiger']


In [ ]:
class TinyTigerCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 4, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc   = nn.Linear(4 * 30 * 30, 1)

    def forward(self, x):
        x = self.relu(self.conv(x))
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(self.fc(x))
        return x


In [ ]:
import os
from PIL import Image

def clean_folder(root):
    bad = 0
    for root_dir, _, files in os.walk(root):
        for f in files:
            path = os.path.join(root_dir, f)
            try:
                with Image.open(path) as img:
                    img.verify()  # checks corruption
            except:
                print("Removing:", path)
                os.remove(path)
                bad += 1
    print("Removed", bad, "bad images")

clean_folder("dataset/train")
clean_folder("dataset/val")


In [ ]:
class TinyTigerCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 4, kernel_size=3)
        self.relu = nn.ReLU()
        self.fc   = nn.Linear(4 * 30 * 30, 1)

    def forward(self, x):
        x = self.relu(self.conv(x))
        x = x.view(x.size(0), -1)
        x = torch.sigmoid(self.fc(x))
        return x


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = TinyTigerCNN().to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(10000):
    model.train()
    loss_sum = 0

    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    print(f"Epoch {epoch+1}: Loss = {loss_sum:.3f}")

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        preds = (model(imgs) > 0.5).int().squeeze()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Validation accuracy:", correct / total)


In [ ]:
from PIL import Image

img = Image.open("tigerog.jpg")
img = transform(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    prob = model(img).item()

print("Tiger probability:", prob)


In [ ]:
model.eval()

In [ ]:
import numpy as np

SCALE = 128  # power of 2 = hardware-friendly

def quantize(tensor):
    return np.round(tensor.cpu().numpy() * SCALE).astype(np.int8)


In [ ]:
conv_w = quantize(model.conv.weight.data)
conv_b = quantize(model.conv.bias.data)

with open("conv_weights.h", "w") as f:
    f.write("#pragma once\n#include <stdint.h>\n\n")

    f.write("static const int8_t CONV_W[4][1][3][3] = {\n")
    for oc in range(4):
        f.write("  {")
        flat = conv_w[oc][0].flatten()
        f.write(",".join(map(str, flat)))
        f.write("},\n")
    f.write("};\n\n")

    f.write("static const int8_t CONV_B[4] = {")
    f.write(",".join(map(str, conv_b)))
    f.write("};\n")


In [ ]:
fc_w = quantize(model.fc.weight.data)
fc_b = quantize(model.fc.bias.data)

with open("fc_weights.h", "w") as f:
    f.write("#pragma once\n#include <stdint.h>\n\n")

    f.write("static const int8_t FC_W[1][3600] = {\n  {")
    f.write(",".join(map(str, fc_w.flatten())))
    f.write("}\n};\n\n")

    f.write("static const int8_t FC_B[1] = {")
    f.write(",".join(map(str, fc_b)))
    f.write("};\n")


In [ ]:
def relu(x):
    return np.maximum(0, x)

def conv2d_int(img, w, b):
    out = np.zeros((4,30,30), dtype=np.int32)
    for oc in range(4):
        for i in range(30):
            for j in range(30):
                s = b[oc]
                for ki in range(3):
                    for kj in range(3):
                        s += img[i+ki][j+kj] * w[oc][0][ki][kj]
                out[oc,i,j] = s >> 7  # scale back
    return out

def fc_int(x, w, b):
    s = np.sum(x.flatten() * w[0]) + b[0]
    return s >> 7


In [ ]:
torch.save(model, "model.pth")
